# ArangoGraphRAG: Workflow Tutorial

This notebook provides a complete guide—using clear Python examples—covering each major step in the GraphRAG pipeline with Arango, including GenAI API usage, service orchestration, document import & conversion, and RAG querying.

## 1. GenAI API

### Setup and Authentication

First, install the required libraries:

In [ ]:
! pip install fitz
! pip install PyMuPDF
! pip install PyPDF2
! pip install markdownify
! pip install docling==2.26.0
! pip install aiohttp
! pip install pymupdf4llm pymupdf_layout ocrmypdf

In [ ]:
# Suppress all warnings to keep output clean
import warnings
warnings.filterwarnings('ignore')

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import requests
requests.packages.urllib3.disable_warnings()

# Suppress DeprecationWarning and other warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter('ignore')

print("✓ Warning suppression enabled")

Import necessary modules and set up environment variables:

In [ ]:
import aiohttp
import asyncio
import json
import base64
import os
import pathlib
import shutil
import pymupdf4llm
from typing import Dict, Optional, List, Tuple
from dotenv import load_dotenv
from docling.document_converter import DocumentConverter

Set up environment variable helper function:

In [ ]:
def get_updated_env_var(env_var_name):
    load_dotenv(dotenv_path="./env_cadence_poc", override=True)
    return os.environ.get(env_var_name, None)

# Initialize JWT token variable
jwt_token = None

Authenticate with the GenAI API and retrieve a JWT token:

In [ ]:
def authenticate():
    global jwt_token
    auth_url = f"{get_updated_env_var('SERVER_URL')}/_open/auth"
    payload = {
        "username": get_updated_env_var("USERNAME"),
        "password": get_updated_env_var("PASSWORD")
    }
    try:
        print("Authenticating with the server...")
        response = requests.post(auth_url, json=payload, verify=False)
        response.raise_for_status()
        jwt_token = response.json().get("jwt")
        if not jwt_token:
            raise ValueError("Authentication response does not contain a token.")
        print("✓ Authentication successful. JWT token retrieved.")
    except Exception as e:
        print(f"✗ Error during authentication: {e}")
        raise

Helper functions for authenticated API requests:

In [ ]:
def send_request(suffix: str, payload: Dict, method: str = "POST") -> Optional[Dict]:
    """Sends an HTTP request with JWT authorization and returns the response."""
    authenticate()
    headers = {
        "Authorization": f"Bearer {jwt_token}",
        "Content-Type": "application/json"
    }
    url = f"{get_updated_env_var('SERVER_URL')}{suffix}"
    
    try:
        print(f"INFO: Sending {method} request to {suffix}")
        response = requests.request(method, url, json=payload, headers=headers, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"ERROR: Error during request to {url}: {e}")
        raise

In [ ]:
async def send_streaming_request_async(
    suffix: str, payload: Dict, method: str = "POST"
) -> Tuple[List[Dict], str]:
    """
    Sends an async HTTP request expecting a streaming response and returns parsed JSON objects.
    
    Args:
        suffix: The URL suffix to send the request to
        payload: The request payload
        method: HTTP method (default: POST)
    
    Returns:
        Tuple of (parsed_json_objects, raw_response_text)
    """
    authenticate()
    headers = {
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
        "Authorization": f"Bearer {jwt_token}",
    }
    url = f"{get_updated_env_var('SERVER_URL')}{suffix}"
    try:
        print(f"Sending {method} async streaming request to {url}")
        
        timeout = aiohttp.ClientTimeout(total=7200)
        async with aiohttp.ClientSession(timeout=timeout) as session:
            async with session.request(method, url, json=payload, headers=headers) as response:
                response.raise_for_status()
                
                # Read the streaming response
                raw_response = ""
                json_objects = []
                
                async for line in response.content:
                    if line:
                        line_str = line.decode("utf-8").strip()
                        raw_response += line_str + "\n"
                        #print(line_str)
                        
                        # Skip empty lines and SSE comments
                        if not line_str or line_str.startswith(":"):
                            continue
                        
                        # Remove SSE data prefix if present
                        if line_str.startswith("data: "):
                            line_str = line_str[6:]
                        
                        # Parse JSON object
                        try:
                            json_obj = json.loads(line_str)
                            json_objects.append(json_obj)
                            print(f"📨 Received response: {json_obj}")
                        except json.JSONDecodeError as e:
                            print(f"Failed to parse JSON: {line_str} - Error: {e}")
                            continue
        return json_objects, raw_response
        
    except aiohttp.ClientError as e:
        print(f"❌ Error during async streaming request to {url}: {e}")
        raise

## 2. Starting a Service

Define the start_service function:

In [ ]:
def start_service(service_name: str, startup_parameters: Dict) -> Optional[Dict]:
    """Start a service using the gen-ai API."""
    body = {
        "service_name": service_name,
        "env": startup_parameters
    }
    return send_request("/gen-ai/v1/service", body)

Create a GenAI project:

In [ ]:
DB_PROJECT_NAME = get_updated_env_var("DB_PROJECT_NAME")
DB_NAME = get_updated_env_var("DB_NAME")

# Create GenAI Project
project_payload = {
    "project_name": DB_PROJECT_NAME,
    "project_db_name": DB_NAME,
    "project_type": "Graph",  # TODO: Enter your project type
    "project_description": "GraphRAG Project"  # TODO: Enter your project description
}

print(f"Creating project '{DB_PROJECT_NAME}' in database '{DB_NAME}'...")
try:
    project_response = send_request("/gen-ai/v1/project", project_payload, "POST")
    print(f"Project '{DB_PROJECT_NAME}' created successfully!")
    print(json.dumps(project_response, indent=2))
except Exception as e:
    error_str = str(e).lower()
    print(f"Project '{DB_PROJECT_NAME}' already exists. Continuing...")

Start the Importer service:

In [ ]:
# Start the GraphRAG Importer service
importer_params = {
    "db_name": get_updated_env_var("DB_NAME"),
    "username": get_updated_env_var("USERNAME"),
    "password": get_updated_env_var("PASSWORD"),
    "chat_model": "gpt-4.1",
    "chat_api_provider": "openai",
    "embedding_api_provider": "openai",
    "chat_api_key": get_updated_env_var("OPENAI_API_KEY"),
    "embedding_api_key": get_updated_env_var("OPENAI_API_KEY"),
    "genai_project_name": get_updated_env_var("DB_PROJECT_NAME"),
    #"profiles": "memory-2gi-cpu-1"
}

importer_response = start_service("arangodb-graphrag-importer", importer_params)
print(json.dumps(importer_response, indent=2))

# Extract service ID
importer_service_id = importer_response["serviceInfo"]["serviceId"].split("-")[-1]
print(f"\nImporter Service ID: {importer_service_id}")



## 3. List of Services

Define list_services function and retrieve running services:

In [ ]:
def list_services() -> Optional[Dict]:
    """List all services using the gen-ai API."""
    return send_request("/gen-ai/v1/list_services", {}, "POST")

services_response = list_services()
print("All Services:")
print(json.dumps(services_response, indent=2))

## 4. Creating an Importer

The Importer service has already been started in section 2. Below is the update function.

### 4.1 Updating an Importer

Define the upgrade_service function:

In [ ]:
def update_service(full_service_id: str, startup_parameters: Dict) -> Optional[Dict]:
    """Update a service by service_id with new configuration, environment variables, or labels.
    
    Args:
        service_id: The unique identifier of the service (path parameter)
        startup_parameters: Dictionary containing new configuration, env vars, or labels
    
    Returns:
        Response from the upgrade service API
    """
    body = {"env": startup_parameters}
    return send_request(f"/gen-ai/v1/service/{full_service_id}", body, "PUT")

Example: Update the Importer service with new parameters:

In [ ]:
# Prepare upgraded parameters
updated_params = importer_params.copy()
updated_params["chat_model"] = "gpt-4o"
updated_params["profiles"] = "memory-8gi-cpu-2"

importer_service_id_full = f"arangodb-graphrag-importer-{importer_service_id}"
print(f"Updating service: {importer_service_id_full}")
response = update_service(importer_service_id_full, updated_params)
print(json.dumps(response, indent=2))

### 4.2 Viewing a Pod in k9s

To view pod details in `k9s`, launch the tool and use the following keyboard shortcuts:

- **Arrow keys**: Navigate through pods
- **ENTER**: View pod details
- **d**: Describe the pod (shows full YAML, status, and events)
- **l**: View logs; press `0`, `1`, `2`... to filter by time range
- **ESC**: Return to previous view
- **SHIFT+F**: Port forward a pod
- **e**: Edit pod configuration
- **CTRL+D**: Delete the pod
- **?**: Show help menu with all available commands

## 5. Preprocessing files

Convert input documents (PDF/TXT) to Markdown using Docling or pymupdf4llm:

In [ ]:
def pdf_to_markdown_docling(pdf_file):
    """Convert PDF to Markdown using Docling"""
    converter = DocumentConverter()
    result = converter.convert(pdf_file)
    output_md_file = pdf_file.replace(".pdf", "_docling.md")
    with open(output_md_file, "w", encoding="utf-8") as md_file:
        md_file.write(result.document.export_to_markdown())
    print(f"Converted {pdf_file} to {output_md_file}")
    return output_md_file

In [ ]:
def pdf_to_markdown_pymupdf4llm(pdf_file, page_chunks=False):
    chunks = pymupdf4llm.to_markdown(pdf_file, page_chunks=page_chunks)
    output_md_file = pdf_file.replace(".pdf", "_pymupdf4llm.md")
    
    # If page_chunks=True, it returns a list
    if isinstance(chunks, list):
        with open(output_md_file, "w", encoding="utf-8") as f:
            for i, chunk in enumerate(chunks, 1):
                f.write(f"## Page {i}\n\n{chunk}\n\n")
    else:
        # If page_chunks=False, it returns a string
        with open(output_md_file, "w", encoding="utf-8") as f:
            f.write(chunks)
    
    return output_md_file

In [ ]:
def encode_file_content(filepath: str) -> Optional[str]:
    """Encodes the file content to Base64."""
    try:
        with open(filepath, "rb") as file:
            encoded_content = base64.b64encode(file.read()).decode("utf-8")
            print(f"✓ Successfully encoded file: {filepath}")
            return encoded_content
    except FileNotFoundError:
        print(f"✗ File not found: {filepath}")
    except Exception as e:
        print(f"✗ Error reading file {filepath}: {e}")
        return None

In [ ]:
def process_folder(
    input_folder: str,
    json_output_file: str,
    output_folder: str = "./markdown_output",
    use_docling_for_pdf: bool = False,
    skip_conversion: bool = False
) -> None:
    """
    Processes a folder full of PDF and/or HTML files, converts them to Markdown,
    and creates a JSON file with information about each converted file.
    
    Args:
        input_folder: Path to the folder containing PDF and/or HTML files (or markdown files if skip_conversion=True)
        json_output_file: Path to the output JSON file
        output_folder: Path to the folder where Markdown files will be saved (default: "./markdown_output")
                       This folder is deleted and recreated each time when skip_conversion=False
        use_docling_for_pdf: If True, uses docling for PDF. If False, uses pymupdf4llm (default)
        skip_conversion: If True, reads markdown files directly from input_folder (output_folder not needed)
    
    Returns:
        List[Dict]: List of files with "name", "content", and "citable_url" keys
    """
    # List to store information about each file
    files_info = []
    
    if skip_conversion:
        # Process existing markdown files from input folder
        input_path = pathlib.Path(input_folder)
        md_files = list(input_path.glob("*.md"))
        
        if not md_files:
            print(f"⚠ No markdown files found in {input_folder}")
            return
        
        print(f"📁 Processing {len(md_files)} existing markdown file(s) from {input_folder}...")
        
        # Process each markdown file
        for md_file_path in md_files:
            md_file_name = md_file_path.name
            # Try to find original file name (remove .md extension and try to match with PDF/HTML)
            base_name = md_file_name.rsplit('.', 1)[0]
            
            # Try to find original file in input folder
            original_name = None
            
            # Try PDF first
            pdf_candidate = input_path / f"{base_name}.pdf"
            if pdf_candidate.exists():
                original_name = pdf_candidate.name
            else:
                # Try HTML
                html_candidate = input_path / f"{base_name}.html"
                if html_candidate.exists():
                    original_name = html_candidate.name
                else:
                    # Try HTM
                    htm_candidate = input_path / f"{base_name}.htm"
                    if htm_candidate.exists():
                        original_name = htm_candidate.name
                    else:
                        # If no original found, use the markdown filename as the name
                        original_name = md_file_name
            
            print(f"\n🔄 Processing existing markdown: {md_file_name}")
            
            try:
                # Encode the markdown file content using encode_file_content
                encoded_content = encode_file_content(str(md_file_path))
                
                if encoded_content:
                    # Create JSON entry for this file
                    file_info = {
                        "name": original_name,
                        "content": encoded_content,
                        "citable_url": ""
                    }
                    files_info.append(file_info)
                    print(f"✓ Information added to JSON for: {original_name}")
                else:
                    print(f"✗ Error encoding content of: {md_file_path}")
            
            except Exception as e:
                print(f"✗ Error processing {md_file_name}: {e}")
                continue
    
    else:
        # Normal conversion flow
        # Delete and recreate output folder
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)
            print(f"🗑️  Deleted existing output folder: {output_folder}")
        
        os.makedirs(output_folder, exist_ok=True)
        print(f"📁 Created output folder: {output_folder}")
        
        # Get all PDF and HTML files from the input folder
        input_path = pathlib.Path(input_folder)
        pdf_files = list(input_path.glob("*.pdf"))
        html_files = list(input_path.glob("*.html")) + list(input_path.glob("*.htm"))
        
        all_files = pdf_files + html_files
        
        if not all_files:
            print(f"⚠ No PDF or HTML files found in {input_folder}")
            return
        
        print(f"📁 Processing {len(all_files)} file(s) from {input_folder}...")
        
        # Process each file
        for file_path in all_files:
            original_name = file_path.name
            file_extension = file_path.suffix.lower()
            
            print(f"\n🔄 Processing: {original_name}")
            
            try:
                # Determine the output markdown file name
                output_md_name = original_name.rsplit('.', 1)[0] + ".md"
                output_md_path = os.path.join(output_folder, output_md_name)
                
                # Convert based on file type
                if file_extension == ".pdf":
                    if use_docling_for_pdf:
                        # Use docling for PDF
                        converter = DocumentConverter()
                        result = converter.convert(str(file_path))
                        markdown_content = result.document.export_to_markdown()
                    else:
                        # Use pymupdf4llm for PDF
                        chunks = pymupdf4llm.to_markdown(str(file_path), page_chunks=False)
                        if isinstance(chunks, list):
                            markdown_content = "\n\n".join(chunks)
                        else:
                            markdown_content = chunks
                elif file_extension in [".html", ".htm"]:
                    # Use docling for HTML
                    converter = DocumentConverter()
                    result = converter.convert(str(file_path))
                    markdown_content = result.document.export_to_markdown()
                else:
                    print(f"⚠ Unsupported format: {file_extension}. Skipping...")
                    continue
                
                # Save the markdown file
                with open(output_md_path, "w", encoding="utf-8") as md_file:
                    md_file.write(markdown_content)
                print(f"✓ Converted to: {output_md_path}")
                
                # Encode the markdown file content using encode_file_content
                encoded_content = encode_file_content(output_md_path)
                
                if encoded_content:
                    # Create JSON entry for this file
                    file_info = {
                        "name": original_name,
                        "content": encoded_content,
                        "citable_url": ""
                    }
                    files_info.append(file_info)
                    print(f"✓ Information added to JSON for: {original_name}")
                else:
                    print(f"✗ Error encoding content of: {output_md_path}")
            
            except Exception as e:
                print(f"✗ Error processing {original_name}: {e}")
                continue
    
    # Save the JSON with all information
    try:
        with open(json_output_file, "w", encoding="utf-8") as json_file:
            json.dump(files_info, json_file, indent=2, ensure_ascii=False)
        print(f"\n✓ JSON saved to: {json_output_file}")
        print(f"✓ Total files processed: {len(files_info)}")
    except Exception as e:
        print(f"✗ Error saving JSON: {e}")
    
    return files_info



In [ ]:
def load_files_from_json(json_file: str) -> List[Dict]:
    """
    Loads the list of files from a JSON file created by process_folder.
    
    Args:
        json_file: Path to the JSON file containing file information
    
    Returns:
        List[Dict]: List of files with "name", "content", and "citable_url" keys
    """
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            files = json.load(f)
        print(f"✓ Loaded {len(files)} file(s) from {json_file}")
        return files
    except FileNotFoundError:
        print(f"✗ JSON file not found: {json_file}")
        return []
    except Exception as e:
        print(f"✗ Error loading JSON file: {e}")
        return []


In [ ]:
# Example usage:
# Process a folder with PDF and HTML files
input_folder = "./markdown_output"
json_output_file = "./files_info.json"    # JSON file with the information

# # Normal mode: converts PDF/HTML to Markdown (output_folder is created/deleted automatically)
# '''
# files = process_folder(
#     input_folder=input_folder,
#     json_output_file=json_output_file,
#     skip_conversion=False
# )
# '''

# Skip conversion mode: reads markdown files directly from input_folder
process_folder(
    input_folder=input_folder,
    json_output_file=json_output_file,
    skip_conversion=True
)

In [ ]:
files = load_files_from_json(json_output_file)

## 6. Importing the Files

Define set of custom entity types:

In [ ]:
custom_entity_types = [
   "PROCESSOR_COMPONENT",
    "REGISTER",
    "INSTRUCTION",
    "HARDWARE_INTERFACE",
    "MEMORY_UNIT",
    "SIGNAL",
    "EXCEPTION_TYPE",
    "ARCHITECTURE_FEATURE",
    "CONFIGURATION",
    "MEASUREMENT",
]

Submit encoded file for Knowledge Graph Generation:

In [ ]:
#Prepare import body
file_content = encode_file_content("markdown_output/openrisc1200_spec_0.7_jp.md")
import_body = {
    "file_name": "openrisc1200_spec_0.7_jp.md",
    "file_content": file_content,
    "chunk_token_size": 1200,  # Maximum tokens per text chunk (default: 1200)
    "enable_chunk_embeddings": True,  # Generate embeddings for chunks (required for vector search)
    "entity_types": custom_entity_types,
    "partition_id": "cadence_1"
}

importer_response = send_request(
    f"/graphrag/importer/{importer_service_id}/v1/import",
    import_body,
    "POST"
)

print(json.dumps(importer_response, indent=2))

In [ ]:
#Prepare import body
file_content = encode_file_content("markdown_output/openrisc1200_spec.md")
import_body = {
    "file_name": "openrisc1200_spec.md",
    "file_content": file_content,
    "chunk_token_size": 1200,  # Maximum tokens per text chunk (default: 1200)
    "enable_chunk_embeddings": True,  # Generate embeddings for chunks (required for vector search)
    "entity_types": custom_entity_types,
    "partition_id": "cadence_2"
}

importer_response = send_request(
    f"/graphrag/importer/{importer_service_id}/v1/import",
    import_body,
    "POST"
)

print(json.dumps(importer_response, indent=2))

In [ ]:
#Prepare import body
file_content = encode_file_content("markdown_output/openrisc1200_supplementary_prm.md")
import_body = {
    "file_name": "openrisc1200_supplementary_prm.md",
    "file_content": file_content,
    "chunk_token_size": 1200,  # Maximum tokens per text chunk (default: 1200)
    "enable_chunk_embeddings": True,  # Generate embeddings for chunks (required for vector search)
    "entity_types": custom_entity_types,
    "partition_id": "cadence_3"
}

importer_response = send_request(
    f"/graphrag/importer/{importer_service_id}/v1/import",
    import_body,
    "POST"
)

print(json.dumps(importer_response, indent=2))

## 7. Creating a Retriever

Start the GraphRAG Retriever service:

In [ ]:
# Start the GraphRAG Retriever service
retriever_params = {
    "db_name": get_updated_env_var("DB_NAME"),
    "username": get_updated_env_var("USERNAME"),
    "password": get_updated_env_var("PASSWORD"),
    "chat_model": "gpt-4.1",
    "chat_api_provider": "openai",
    "embedding_api_provider": "openai",
    "chat_api_key": get_updated_env_var("OPENAI_API_KEY"),
    "embedding_api_key": get_updated_env_var("OPENAI_API_KEY"),
    "genai_project_name": get_updated_env_var("DB_PROJECT_NAME"),
}

retriever_response = start_service("arangodb-graphrag-retriever", retriever_params)
print(json.dumps(retriever_response, indent=2))

# Extract service ID
retriever_service_id = retriever_response["serviceInfo"]["serviceId"].split("-")[-1]
print(f"\nRetriever Service ID: {retriever_service_id}")

Update retriever service params:

In [ ]:
# Prepare updated parameters
updated_params = retriever_params.copy()
updated_params["chat_model"] = "gpt-4o"

retriever_service_id_full = f"arangodb-graphrag-retriever-{retriever_service_id}"
print(f"Updating service: {retriever_service_id_full}")
response = update_service(retriever_service_id_full, updated_params)
print(json.dumps(response, indent=2))

## 7.1 Getting a Streamed Response

Query the knowledge graph with streaming response for real-time updates:

In [ ]:
async def send_streaming_query(url: str, payload: Dict) -> Tuple[List[Dict], str]:
    """Sends an async HTTP request expecting a streaming response (SSE-like)."""
    if not jwt_token:
        raise ValueError("JWT token is not set. Please authenticate first.")
    
    authenticate()
    headers = {
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
        "Authorization": f"Bearer {jwt_token}"
    }
    
    timeout = aiohttp.ClientTimeout(total=7200)
    raw_response = ""
    json_objects: List[Dict] = []
    
    try:
        async with aiohttp.ClientSession(timeout=timeout, connector=aiohttp.TCPConnector(ssl=False)) as session:
            async with session.post(url, json=payload, headers=headers) as response:
                response.raise_for_status()
                async for line in response.content:
                    if not line:
                        continue
                    line_str = line.decode("utf-8", errors="replace").strip()
                    raw_response += line_str
                    if not line_str:
                        continue
                    if line_str.startswith(":"):
                        continue
                    if line_str.startswith("data: "):
                        line_str = line_str[6:]
                    try:
                        obj = json.loads(line_str)
                        json_objects.append(obj)
                        print(f"✓ Received: {obj}")
                    except json.JSONDecodeError:
                        continue
                return json_objects, raw_response
    except aiohttp.ClientError as e:
        print(f"ERROR: Error during streaming: {e}")
        raise

In [ ]:
# Example streamed query
query_body = {
    "query": "In Z-Tunnel 2.0, how does the MTU configuration differ between DTLS and TLS modes?",
    "query_type": 3,  # 1 for global, 2 for local/deep-search, 3 instant search
}

stream_url = f"{get_updated_env_var('SERVER_URL')}/graphrag/retriever/{retriever_service_id}/v1/graphrag-query-stream"

print(f"Streaming query to: {stream_url}")
print(f"Query: {query_body['query']}")

try:
    asyncio.get_running_loop()
    events, raw = await send_streaming_query(stream_url, query_body)
except RuntimeError:
    events, raw = asyncio.run(send_streaming_query(stream_url, query_body))

print(f"✓ Events received: {len(events)}")

## 7.2 Getting a Full Response

Query the knowledge graph and get the complete response in one request:

In [ ]:
# Full response query
query_body = {
    "query": "In Z-Tunnel 2.0, how does the MTU configuration differ between DTLS and TLS modes?",
    "query_type": 2,  # 1 for global, 2 for local/deep-search, 3 instant search
}

full_response = send_request(
    f"/graphrag/retriever/{retriever_service_id}/v1/graphrag-query",
    query_body,
    "POST"
)

print("Full Response:")
print(json.dumps(full_response, indent=2))
print("\nResult:")
print(full_response.get("result", ""))

## Cleanup: Stop Services

Remember to stop services when finished to free up resources:

In [ ]:
def stop_service(service_id: str) -> Optional[Dict]:
    """Stop a service using the gen-ai API."""
    body = {"service_id": service_id}
    return send_request(f"/gen-ai/v1/service/{service_id}", body, "DELETE")

Stop Importer service:

In [ ]:
importer_stop_response = stop_service(f"arangodb-graphrag-importer-{importer_service_id}")
print(json.dumps(importer_stop_response, indent=2))

Stop Retriever service:

In [ ]:
retriever_stop_response = stop_service(f"arangodb-graphrag-retriever-{retriever_service_id}")
print(json.dumps(retriever_stop_response, indent=2))

---

## Summary

This notebook provides a complete, production-ready workflow for using Arango GraphRAG services:

1. **GenAI API** – Authentication and request handling
2. **Starting Services** – Launch Importer and Retriever
3. **Service Management** – List, upgrade, and stop services
4. **Document Processing** – Convert PDFs to Markdown
5. **Knowledge Graph Import** – Upload documents for processing
6. **Querying** – Retrieve streamed, full, or context-only responses
7. **Kubernetes Integration** – Use k9s for pod monitoring

All code is fully functional and well-documented for easy integration into your projects.